# Mapping of page IDs to properties we are interested in

In [ ]:
map_query = '''
WITH all_pages AS (
    SELECT page_id
    FROM wmf.mediawiki_history
    WHERE snapshot = ""2019-11""
    AND wiki_db = ""commonswiki""
    AND event_entity = ""revision""
    AND event_type = ""create""
    AND event_timestamp >= ""2019-07-31""
    AND event_timestamp < ""2019-12-01""
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
),
label_pages AS (
    SELECT page_id, 1 AS has_label
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND wiki_db = "commonswiki"
    AND event_entity = "revision"
    AND event_type = "create"
    AND event_timestamp >= "{start_date}"
    AND event_timestamp < "{end_date}"
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
    AND event_comment REGEXP "^...wbsetlabel-(add|set):"
),
depicts_pages AS (
    SELECT page_id, 1 AS has_depicts
    FROM wmf.mediawiki_history
    WHERE snapshot = "2019-11"
    AND wiki_db = "commonswiki"
    AND event_entity = "revision"
    AND event_type = "create"
    AND event_timestamp >= "2019-07-31"
    AND event_timestamp < "2019-12-01"
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
    AND (event_comment REGEXP "^...wbcreateclaim-create:"
         OR event_comment REGEXP "^...wbeditentity-update:"
         OR event_comment REGEXP "^...wbsetclaim-create:"
         OR event_comment REGEXP "^...wbsetclaim-update:"
    )
    AND event_comment LIKE "%P180%"
)

-- add all the properties here...

SELECT all_pages.page_id,
  has_label, has_depicts
FROM all_pages
LEFT JOIN label_pages
ON page_id
LEFT JOIN depicts_pages
ON page_id
WHERE (has_label IS NOT NULL
       OR has_depicts IS NOT NULL)

'''